# Async Playwright Utilities Example

This notebook demonstrates the usage of the async Playwright utilities in `utils/playwright_utils.py`.

## Setup and Imports

In [ ]:
import asyncio
import importlib
import sys
from pathlib import Path

# Remove utils from cache to force full reload
for module in list(sys.modules.keys()):
    if module.startswith('utils'):
        del sys.modules[module]

# Now import fresh
from playwright_utils import (
    BrowserManager,
    PageHelper,
    quick_screenshot,
    extract_page_data,
    parse_cookie_string,
    load_cookies_from_file,
)

## Example 1: Quick Screenshot

The simplest way to take a screenshot of a webpage.

In [28]:
# Take a quick screenshot of a webpage
success = await quick_screenshot(
    url="https://stockanalysis.com/",
    output_path="screenshot_example.png",
    headless=False,
    full_page=True,
)

print(f"Screenshot {'succeeded' if success else 'failed'}")

Screenshot succeeded


## Example 2: Extract Page Data

Extract specific text content from a webpage using CSS selectors.

In [4]:
# Extract data from example.com
data = await extract_page_data(
    url="https://www.popularmechanics.com/riddles-logic-puzzles/",
    selectors={
        "title": "h1",
        "description": "p",
    },
)

print("Extracted data:")
for key, value in data.items():
    print(f"  {key}: {value}")

Extracted data:
  title: Riddles
  description: A Part of Hearst Digital Media


## Example 3: Using BrowserManager with Context Manager

For more control, use the BrowserManager with async context managers.

## Example 3.5: Using Cookies from File

Load cookies from a file in header string format and use them in your browser session.

## Example 4: Advanced Page Interactions

Demonstrate form filling, clicking, and waiting for elements.

In [10]:
# Advanced interactions example
async with BrowserManager(headless=False, slow_mo=100) as manager:
    async with manager.new_page_with_cookies_from_file("cookies.txt", domain="stockanalysis.com") as page:
        helper = PageHelper(page)
        
        # Navigate to a search page (example: DuckDuckGo)
        if await helper.navigate("https://stockanalysis.com/"):
            # Wait for search input
            if await helper.wait_for_element("input[name='q']"):
                # Fill search input
                await helper.fill_input("input[name='q']", "MSFT")
                print("Filled search input")
                
                
                # Click search button
                await helper.press_key(selector="input[name='q']", key="Enter")
                print("Pressed Enter to search")
                # Take screenshot of results
                await helper.screenshot("search_results.png")
                print("Screenshot of results saved!")

Filled search input
Pressed Enter to search
Screenshot of results saved!


## Example 5: JavaScript Execution

Execute custom JavaScript in the page context.

In [11]:
# Execute JavaScript in the page
async with BrowserManager(headless=True) as manager:
    async with manager.new_page() as page:
        helper = PageHelper(page)
        
        if await helper.navigate("https://example.com"):
            # Get page dimensions
            dimensions = await helper.evaluate_script(
                """() => {
                    return {
                        width: document.documentElement.scrollWidth,
                        height: document.documentElement.scrollHeight,
                        title: document.title
                    };
                }"""
            )
            
            print(f"Page dimensions: {dimensions}")

Page dimensions: {'width': 1280, 'height': 720, 'title': 'Example Domain'}


## Example 6: Multiple Browser Types

Use different browser engines (Chromium, Firefox, WebKit).

In [ ]:
# Test with different browsers
browsers = ["chromium", "firefox", "webkit"]

for browser_type in browsers:
    print(f"\nTesting with {browser_type}...")
    success = await quick_screenshot(
        url="https://example.com",
        output_path=f"screenshot_{browser_type}.png",
        browser_type=browser_type,
    )
    print(f"  {browser_type}: {'✓ Success' if success else '✗ Failed'}")

## Cleanup

The context managers automatically handle cleanup, but here's how you can verify files were created:

In [ ]:
# List all screenshots created
screenshots = list(Path(".").glob("*.png"))
print(f"\nScreenshots created ({len(screenshots)}):")
for screenshot in screenshots:
    print(f"  - {screenshot.name}")

In [2]:
import pandas as pd
import numpy as np
from utils.pd_parser import parse_percentage, parse_row_percentages

csv_path = 'data/ANET/income.csv'
df = pd.read_csv(csv_path)
rows_parsed = parse_row_percentages('Net Income Growth', df)
display(rows_parsed)

[-26.2, 32.51, 60.84, 54.34, 36.64]